# 🇰🇷🇺🇸 한국어 → 영어 기계번역 프로젝트
한국어 문장을 영어로 번역하는 NMT 모델을 직접 구현하고 비교한다.

구현한 모델
1) 기본 Seq2Seq (GRU Encoder-Decoder, Attention 없음)
2) Seq2Seq + Bahdanau Attention (+ SentencePiece 토크나이징)

평가 방법
- 샘플 번역 출력(정성 평가)
- BLEU 점수(정량 평가)


◻️ 환경설치 & 임포트

In [ ]:
# sentencepiece: 서브워드 토크나이저 학습/사용
# nltk: 토큰화 및 BLEU 계산에 사용 가능
# sacrebleu: BLEU 계산을 좀 더 안정적으로 하고 싶을 때 옵션

!pip -q install sentencepiece nltk sacrebleu

import os
import json
import random
import math
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import sentencepiece as spm

# BLEU 계산용 (nltk or sacrebleu 중에 선택가능)
import sacrebleu

# 재현성: 같은 랜덤 시드면 결과가 "비슷하게" 나오도록 42로 고정
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


◻️ 구글드라이브 마운트 + 경로 설정

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

DATA_DIR = "/content/drive/MyDrive/일생생활및구어체_한영"

TRAIN_PATH = os.path.join(DATA_DIR, "일상생활및구어체_한영_train_set.json")
VALID_PATH = os.path.join(DATA_DIR, "일상생활및구어체_한영_valid_set.json")

print("TRAIN_PATH:", TRAIN_PATH)
print("VALID_PATH:", VALID_PATH)
print("exists(train)?", os.path.exists(TRAIN_PATH))
print("exists(valid)?", os.path.exists(VALID_PATH))


◻️ 데이터 로드 설명
- JSON 파일은 파이썬에서 바로 학습에 못 쓰고,(한국어 문장, 영어 문장) 쌍 형태로 정리해야 한다.

그래서:
- JSON을 읽어서
- ("ko", "mt") 키를 꺼내고
- (ko_sentence, en_sentence) 리스트로 만든다


In [ ]:
def load_json_pairs(path: str) -> List[Tuple[str, str]]:
    """
    왜 함수로 만들까?
    - 같은 로직을 train/valid에 반복 적용해야 한다
    - 함수로 만들면 '재사용' + '실수 감소'

    반환값:
    - [(ko문장, en문장), (ko문장, en문장), ...]
    """
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    pairs = []
    for item in data:
        ko = item["ko"].strip()
        en = item["mt"].strip()
        pairs.append((ko, en))
    return pairs

train_pairs = load_json_pairs(TRAIN_PATH)
valid_pairs = load_json_pairs(VALID_PATH)

print("train size:", len(train_pairs))
print("valid size:", len(valid_pairs))
print("\n샘플 1개:")
print("KO:", train_pairs[0][0])
print("EN:", train_pairs[0][1])


◻️ SentencePiece 학습(토크나이저)
- 단어 기반 토큰화는 OOV(사전에 없는 단어)에 취약하다.
SentencePiece는 "서브워드" 단위로 쪼개서 OOV를 줄여준다.

이번 실험은
- vocab_size = 8000
- model_type = unigram
- pad_id=0, bos_id=1, eos_id=2, unk_id=3


In [ ]:
SPM_DIR = os.path.join(DATA_DIR, "spm_models")
os.makedirs(SPM_DIR, exist_ok=True)

KO_CORPUS = os.path.join(SPM_DIR, "train_ko.txt")
EN_CORPUS = os.path.join(SPM_DIR, "train_en.txt")

KO_MODEL_PREFIX = os.path.join(SPM_DIR, "spm_ko")
EN_MODEL_PREFIX = os.path.join(SPM_DIR, "spm_en")

KO_MODEL_PATH = KO_MODEL_PREFIX + ".model"
EN_MODEL_PATH = EN_MODEL_PREFIX + ".model"

def write_corpus(pairs: List[Tuple[str, str]], ko_path: str, en_path: str):
    """
    SentencePieceTrainer는 '텍스트 파일'을 input으로 받는다.
    그래서 JSON → 텍스트 파일(한 줄에 한 문장)로 변환이 필요하다.
    """
    with open(ko_path, "w", encoding="utf-8") as fko, open(en_path, "w", encoding="utf-8") as fen:
        for ko, en in pairs:
            fko.write(ko.replace("\n", " ") + "\n")
            fen.write(en.replace("\n", " ") + "\n")

# 말뭉치 파일이 없으면 생성
if (not os.path.exists(KO_CORPUS)) or (not os.path.exists(EN_CORPUS)):
    write_corpus(train_pairs, KO_CORPUS, EN_CORPUS)
    print("corpus files created")

def train_spm_if_needed(corpus_path: str, model_prefix: str, vocab_size: int = 8000):
    """
    모델 파일이 없으면 학습하고, 있으면 넘어간다.
    (Colab 재실행할 때 매번 학습하면 시간 낭비라서)
    """
    model_path = model_prefix + ".model"
    if os.path.exists(model_path):
        print("skip training:", model_path)
        return

    spm.SentencePieceTrainer.train(
        input=corpus_path,
        model_prefix=model_prefix,
        vocab_size=vocab_size,
        model_type="unigram",
        character_coverage=1.0,
        pad_id=0, unk_id=3, bos_id=1, eos_id=2,
        # user_defined_symbols는 "문자열 심볼"용인데,
        # 여기서는 id를 명확히 지정했으니 필수는 아님.
    )
    print("trained:", model_path)

train_spm_if_needed(KO_CORPUS, KO_MODEL_PREFIX, vocab_size=8000)
train_spm_if_needed(EN_CORPUS, EN_MODEL_PREFIX, vocab_size=8000)

# 로드
sp_ko = spm.SentencePieceProcessor()
sp_en = spm.SentencePieceProcessor()
sp_ko.load(KO_MODEL_PATH)
sp_en.load(EN_MODEL_PATH)

PAD_ID = 0
BOS_ID = 1
EOS_ID = 2
UNK_ID = 3

print("KO vocab size:", sp_ko.get_piece_size())
print("EN vocab size:", sp_en.get_piece_size())


◻️ 데이터셋 & 패딩
- 왜 Dataset / DataLoader를 쓰나?  
    - 딥러닝 학습은 "배치(batch)" 단위로 데이터를 흘려보내는 게 기본이다.
    - 문장 길이는 제각각이라 그대로 batch로 묶으면 텐서 크기가 안 맞는다.

그래서:
- 각 문장을 id 시퀀스로 바꾸고
- batch에서 가장 긴 길이에 맞춰 PAD_ID로 패딩해서
- (batch_size, seq_len) 형태 텐서를 만든다


In [ ]:
def encode_pair(ko: str, en: str, max_len: int):
    """
    왜 BOS/EOS를 붙이나?
    - BOS: 디코더가 "시작"을 알게 해주는 토큰
    - EOS: 디코더가 "끝"을 알게 해주는 토큰
    """
    ko_ids = [BOS_ID] + sp_ko.encode_as_ids(ko) + [EOS_ID]
    en_ids = [BOS_ID] + sp_en.encode_as_ids(en) + [EOS_ID]

    # 너무 길면 자르기(학습 안정성)
    ko_ids = ko_ids[:max_len]
    en_ids = en_ids[:max_len]

    return ko_ids, en_ids

# max_len을 데이터에서 대충 정해주면 (너무 짧거나 너무 길거나) 학습이 흔들릴 수 있음
# 여기서는 원본 노트북의 흐름처럼 "적당한 상수"로 둔다.
MAX_LEN = 60

class TranslationDataset(Dataset):
    def __init__(self, pairs: List[Tuple[str, str]]):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        ko, en = self.pairs[idx]
        src_ids, tgt_ids = encode_pair(ko, en, MAX_LEN)
        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

def collate_fn(batch):
    """
    batch: [(src_tensor, tgt_tensor), (src_tensor, tgt_tensor), ...]

    왜 여기서 패딩을 하나?
    - DataLoader가 batch를 만들 때 길이가 다른 텐서는 stack이 안 됨
    - 그래서 batch 내부 최대 길이에 맞춰 PAD로 채워서 stack 가능하게 함
    """
    src_list, tgt_list = zip(*batch)

    src_lens = torch.tensor([len(x) for x in src_list], dtype=torch.long)
    tgt_lens = torch.tensor([len(x) for x in tgt_list], dtype=torch.long)

    src_max = max(src_lens).item()
    tgt_max = max(tgt_lens).item()

    def pad_to(x, max_len):
        pad_len = max_len - len(x)
        if pad_len <= 0:
            return x
        return torch.cat([x, torch.full((pad_len,), PAD_ID, dtype=torch.long)])

    src_padded = torch.stack([pad_to(x, src_max) for x in src_list], dim=0)
    tgt_padded = torch.stack([pad_to(x, tgt_max) for x in tgt_list], dim=0)

    return src_padded, src_lens, tgt_padded, tgt_lens

BATCH_SIZE = 64
train_loader = DataLoader(TranslationDataset(train_pairs), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(TranslationDataset(valid_pairs), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

batch = next(iter(train_loader))
print("src:", batch[0].shape, "src_lens:", batch[1].shape)
print("tgt:", batch[2].shape, "tgt_lens:", batch[3].shape)


◻️ Attention 모델 정의
- 기본 Seq2Seq는 Encoder가 마지막 hidden 하나로 모든 정보를 압축해야 해서,
문장이 길어지면 번역 품질이 떨어지기 쉽다.

Bahdanau Attention은:
- 디코더가 매 타임스텝마다
- encoder output 전체를 보고
- "지금 출력할 단어에 중요한 입력 위치"를 가중치로 뽑는다.

즉, 디코더가 "필요할 때마다" 입력 문장을 다시 참조하는 구조


▫️Encoder / Attention / Decoder / Seq2Seq

In [ ]:
class EncoderBiGRU(nn.Module):
    """
    BiGRU(양방향 GRU)를 쓰는 이유(WHY):
    - forward 방향만 보면 "과거"만 참고
    - backward까지 보면 "미래 문맥"도 같이 반영
    - 번역에서는 입력 문장 전체 문맥이 중요해서 Bi가 종종 유리

    문법 포인트:
    - nn.Module: PyTorch 모델의 기본 틀
    - super().__init__(): 부모 초기화 호출(파이썬 OOP 기본)
    """
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=PAD_ID)
        self.gru = nn.GRU(
            embed_dim, hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )

    def forward(self, src, src_lens):
        # src: (B, L)
        emb = self.embedding(src)  # (B, L, E)

        # pack: 패딩 구간은 RNN이 학습/계산하지 않게 하는 테크닉(WHY: 효율 + 성능)
        packed = nn.utils.rnn.pack_padded_sequence(emb, src_lens.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, hidden = self.gru(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)  # (B, L, 2H)

        # hidden: (num_layers*2, B, H)  -> 양방향이라 *2
        return out, hidden

class BahdanauAttention(nn.Module):
    """
    score(h_t, enc_out_s)를 구해서 softmax로 가중치를 만든 뒤,
    enc_out의 가중합(context vector)을 만든다.
    """
    def __init__(self, enc_dim, dec_dim, attn_dim):
        super().__init__()
        self.W = nn.Linear(enc_dim, attn_dim, bias=False)
        self.U = nn.Linear(dec_dim, attn_dim, bias=False)
        self.v = nn.Linear(attn_dim, 1, bias=False)

    def forward(self, decoder_hidden, encoder_outputs, src_mask):
        """
        decoder_hidden: (B, dec_dim)
        encoder_outputs: (B, L, enc_dim)
        src_mask: (B, L)  -> PAD가 아닌 위치 True/False
        """
        # (B, L, attn_dim)
        score = self.v(torch.tanh(self.W(encoder_outputs) + self.U(decoder_hidden).unsqueeze(1))).squeeze(-1)
        score = score.masked_fill(~src_mask, -1e9)  # PAD는 거의 0확률로 만들기
        attn_weights = torch.softmax(score, dim=1)  # (B, L)

        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)  # (B, enc_dim)
        return context, attn_weights

class AttnDecoderGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim, enc_dim, dec_dim, attn_dim, num_layers=1, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=PAD_ID)
        self.attn = BahdanauAttention(enc_dim, dec_dim, attn_dim)

        # 디코더 입력 = (현재 입력 토큰 임베딩 + context)
        self.gru = nn.GRU(embed_dim + enc_dim, dec_dim, num_layers=num_layers, batch_first=True,
                          dropout=dropout if num_layers > 1 else 0.0)

        self.fc_out = nn.Linear(dec_dim + enc_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_tokens, decoder_hidden, encoder_outputs, src_mask):
        """
        input_tokens: (B,) - 현재 타임스텝의 입력 토큰 id
        decoder_hidden: (num_layers, B, dec_dim)
        """
        emb = self.dropout(self.embedding(input_tokens)).unsqueeze(1)  # (B,1,E)

        # attention은 보통 "마지막 레이어 hidden"을 사용
        dec_h_last = decoder_hidden[-1]  # (B, dec_dim)
        context, attn_w = self.attn(dec_h_last, encoder_outputs, src_mask)  # (B, enc_dim)

        rnn_input = torch.cat([emb, context.unsqueeze(1)], dim=-1)  # (B,1,E+enc_dim)
        out, new_hidden = self.gru(rnn_input, decoder_hidden)       # out: (B,1,dec_dim)

        out = out.squeeze(1)  # (B, dec_dim)
        logits = self.fc_out(torch.cat([out, context], dim=-1))  # (B, vocab)
        return logits, new_hidden, attn_w

class Seq2SeqAttn(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lens, tgt, teacher_forcing_ratio=0.5):
        """
        teacher_forcing_ratio를 쓰는 이유(WHY):
        - 학습 초반에 모델이 예측을 잘 못하니까,
          정답 토큰을 다음 입력으로 넣어주면 학습이 안정적
        - 다만 너무 많이 쓰면 실제 추론(정답이 없는 상황)에 약해질 수 있음
        """
        B = src.size(0)
        T = tgt.size(1)
        vocab_size = self.decoder.fc_out.out_features

        src_mask = (src != PAD_ID)  # (B, L)

        enc_out, enc_hidden = self.encoder(src, src_lens)

        # BiGRU hidden을 디코더 차원으로 맞춰주는 방식은 여러가지가 있음.
        # 여기서는 간단히 "마지막 레이어의 forward/backward를 concat 후 선형 변환" 같은 방법을 쓰기도 한다.
        # 원본 노트북 흐름을 최대한 단순화해서, 디코더 초기 hidden을 0으로 시작(가장 안전한 베이스라인)한다.
        dec_hidden = torch.zeros(1, B, 256, device=src.device)

        outputs = torch.zeros(B, T, vocab_size, device=src.device)

        input_tok = tgt[:, 0]  # BOS

        for t in range(1, T):
            logits, dec_hidden, _ = self.decoder(input_tok, dec_hidden, enc_out, src_mask)
            outputs[:, t] = logits

            use_tf = random.random() < teacher_forcing_ratio
            top1 = logits.argmax(dim=-1)  # (B,)
            input_tok = tgt[:, t] if use_tf else top1

        return outputs


▫️학습/검증 루프
- CrossEntropyLoss는 "분류" 손실이다.
- 번역은 매 타임스텝마다 다음 토큰을 "분류"하는 문제로 바꿀 수 있다.
  (예: "다음 토큰이 vocab에서 무엇인가?")

그래서:
- decoder output logits: (B, T, vocab)
- target: (B, T)
를
- (B*T, vocab) vs (B*T)
로 펼쳐서(loss 계산을 쉽게) 학습한다.


▫️train/eval 함수

In [ ]:
def train_epoch(model, loader, optimizer, criterion, teacher_forcing_ratio=0.5):
    model.train()
    total_loss = 0.0

    for src, src_lens, tgt, tgt_lens in loader:
        src, src_lens, tgt = src.to(device), src_lens.to(device), tgt.to(device)

        optimizer.zero_grad()
        out = model(src, src_lens, tgt, teacher_forcing_ratio=teacher_forcing_ratio)
        # out: (B, T, vocab)

        # 타임스텝 0(BOS)은 예측 대상에서 제외하는 경우가 일반적
        out = out[:, 1:].contiguous()
        tgt_y = tgt[:, 1:].contiguous()

        loss = criterion(out.view(-1, out.size(-1)), tgt_y.view(-1))
        loss.backward()

        # 기울기 폭발 방지(WHY: RNN은 폭발이 잘 일어남)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

@torch.no_grad()
def eval_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0.0

    for src, src_lens, tgt, tgt_lens in loader:
        src, src_lens, tgt = src.to(device), src_lens.to(device), tgt.to(device)

        out = model(src, src_lens, tgt, teacher_forcing_ratio=0.0)
        out = out[:, 1:].contiguous()
        tgt_y = tgt[:, 1:].contiguous()

        loss = criterion(out.view(-1, out.size(-1)), tgt_y.view(-1))
        total_loss += loss.item()

    return total_loss / len(loader)


▫️학습 실행 + best 저장

In [ ]:
# 원본 노트북 하이퍼파라미터(네 파일 기준)
BATCH_SIZE = 64
embed_dim = 256
hidden_dim = 256
dropout = 0.1
lr = 0.001
num_epochs = 30

encoder = EncoderBiGRU(vocab_size=sp_ko.get_piece_size(), embed_dim=embed_dim, hidden_dim=hidden_dim, dropout=dropout).to(device)
decoder = AttnDecoderGRU(vocab_size=sp_en.get_piece_size(), embed_dim=embed_dim,
                         enc_dim=hidden_dim*2, dec_dim=hidden_dim, attn_dim=hidden_dim,
                         dropout=dropout).to(device)

model = Seq2SeqAttn(encoder, decoder).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)
optimizer = optim.Adam(model.parameters(), lr=lr)

BEST_PATH = os.path.join(DATA_DIR, "best_seq2seq_attn.pt")

best_val = float("inf")

for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, teacher_forcing_ratio=0.5)
    val_loss = eval_epoch(model, valid_loader, criterion)

    saved = ""
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), BEST_PATH)
        saved = " | Saved"

    print(f"Epoch: {epoch}/{num_epochs} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f}{saved}")

print("best_val_loss:", best_val)
print("best_model_path:", BEST_PATH)


▫️추론(번역)함수
- 학습 때는 정답 토큰을 다음 입력으로 넣어주는 Teacher Forcing을 썼지만,
- 실제 번역(추론)에서는 정답이 없으므로 "내가 예측한 토큰"을 다음 입력으로 사용한다.

- 이때 기본은 greedy decoding(argmax)인데, BLEU를 올리려면 beam search로 바꾸는 경우가 많다.


▫️translate 함수 + 샘플 출력

In [ ]:
@torch.no_grad()
def translate(model, ko_sentence: str, max_len: int = 60):
    model.eval()

    src_ids, _ = encode_pair(ko_sentence, "dummy", max_len)
    src = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)  # (1, L)
    src_lens = torch.tensor([len(src_ids)], dtype=torch.long).to(device)

    src_mask = (src != PAD_ID)

    enc_out, _ = model.encoder(src, src_lens)

    # 디코더 초기 hidden(여기선 0 시작 베이스라인)
    dec_hidden = torch.zeros(1, 1, 256, device=device)

    input_tok = torch.tensor([BOS_ID], device=device)

    out_ids = []
    for _ in range(max_len):
        logits, dec_hidden, _ = model.decoder(input_tok, dec_hidden, enc_out, src_mask)
        top1 = logits.argmax(dim=-1).item()

        if top1 == EOS_ID:
            break
        out_ids.append(top1)
        input_tok = torch.tensor([top1], device=device)

    return sp_en.decode_ids(out_ids)

# best 모델 로드 후 샘플 테스트
model.load_state_dict(torch.load(BEST_PATH, map_location=device))

for i in range(5):
    ko, en = random.choice(valid_pairs)
    pred = translate(model, ko)
    print(f"\n[입력] {ko}")
    print(f"[정답] {en}")
    print(f"[예측] {pred}")


▫️BLEU 계산

In [ ]:
@torch.no_grad()
def compute_bleu(model, pairs, num_samples=1000):
    """
    sacrebleu를 사용하면 토큰화/정규화가 비교적 안정적이라 실험 비교가 편하다.
    """
    model.eval()
    samples = random.sample(pairs, min(num_samples, len(pairs)))

    preds = []
    refs = []

    for ko, en in samples:
        pred = translate(model, ko)
        preds.append(pred)
        refs.append(en)

    bleu = sacrebleu.corpus_bleu(preds, [refs]).score / 100.0
    return bleu

bleu = compute_bleu(model, valid_pairs, num_samples=1000)
print(f"최종 BLEU 점수: {bleu:.4f}")


## 🟨 결론 및 회고 (실험 결과 종합)

이번 미션은 한국어 → 영어 기계번역을 목표로, Seq2Seq 계열 모델을 단계적으로 개선하면서
(1) 학습은 되는 것처럼 보이는데 실제 번역 품질은 왜 안 나오는지를 직접 확인한 프로젝트였다.

핵심은:
- 모델 구조(Seq2Seq vs Attention)도 중요하지만,
- 실제 성능은 데이터 전처리 + 토크나이징(어휘 문제) + 학습/추론 방식(Teacher Forcing, decoding)이 훨씬 크게 좌우했다.

---

### 1) 실험 1: 기본 Seq2Seq (단어 기반 토크나이저)
- 학습 로그상으로는 Loss가 계속 내려가며 학습이 “되는 것처럼” 보였다.
  - 예: 30 epoch 기준 Train Loss가 약 0.76 → 0.27 수준까지 감소
- 하지만 실제 번역 결과(정성 평가)는 다음 문제가 반복됐다.
  - UNK(⁇) 토큰이 자주 등장 → 단어 기반 vocab이 커버 못 하는 단어가 많음
  - 문장 구조가 무너지고, 의미가 맞지 않는 출력이 나옴
  - 학습 데이터의 특수문자/따옴표/기호가 남아있으면 출력이 더 흔들림

정리:
- Loss가 내려간다 = 번역이 잘 된다가 아니었다.
- 특히 단어 기반 토크나이저는 OOV(사전에 없는 단어) 문제 때문에 번역에서 바로 한계가 드러났다.

---

### 2) 실험 2: Seq2Seq + Attention (단어 기반 토크나이저 유지)
- Attention을 붙이면 이론적으로 "어떤 입력 단어를 참고해야 하는지"를 더 잘 배우는 구조다.
- 실제로 Loss는 더 좋아졌다.
  - 예: 30 epoch 기준 Train Loss가 대략 0.74 → 0.21 수준까지 감소
- 하지만 번역 출력은 기대만큼 좋아지지 않았다.
  - 기본 토크나이저/전처리의 한계가 그대로 남아있어서,
  - Attention이 있어도 “제대로 된 단어를 못 읽고(UNK), 제대로 된 단어를 못 내보내는” 문제가 유지됐다.
- 이 구간에서 BLEU 평가가 사실상 의미가 없을 정도로 낮게 나왔다.
  - 예: BLEU score: 0.0000 출력이 확인됨

정리:
- Attention은 엔진 성능을 올리는 업그레이드에 가깝다.
- 그런데 데이터와 토크나이징/전처리가 엉망이면 엔진만 좋아져서는 결과가 안 나온다.

---

### 3) 실험 3: SentencePiece(Subword) + 학습 안정화(Teacher Forcing 조절) + Beam Search
여기서부터 “번역 모델답게” 성능이 올라가기 시작했다.

적용한 개선 포인트:
- SentencePiece(Subword) 기반 토크나이저
  - 단어 단위가 아니라 조각(subword) 단위로 처리 → OOV/UNK 급감
- Teacher Forcing 비율을 점진적으로 감소
  - 초반엔 정답을 많이 보여주며 안정적으로 학습시키고,
  - 후반엔 정답 의존을 줄여서 "실전(추론)"에 적응시키는 방식
  - 실제 로그에서도 TF가 1.00 → 0.30까지 내려가며 학습됨
- Beam Search 디코딩
  - greedy(한 번에 최고만 고르기)보다 문장 품질이 좋아질 가능성이 큼

결과(정량):
- 학습 로그 예시:
  - Epoch 1: Train Loss 5.509, Val Loss 6.000
  - Epoch 30: Train Loss 2.721, Val Loss 5.267
- BLEU 점수(검증셋):
  - Validation BLEU: 0.1298

정리:
- 이 실험에서 “이제야 번역이 성능으로 측정 가능한 수준”까지 올라왔다.
- 특히 BLEU가 0에서 **0.1298로 올라간 변화 자체가** 토크나이징/추론 방식의 영향력을 보여준다.

---

## 이번 미션에서 얻은 교훈

1) 토크나이저/어휘가 성능의 상한선을 결정한다
- 단어 기반 vocab은 현실 데이터에서 OOV가 너무 많이 발생한다.
- 번역에서 OOV는 곧바로 UNK 또는 의미 붕괴로 이어진다.
- Subword(SentencePiece)는 이 문제를 구조적으로 완화한다.

2) Loss는 "학습 중 성능"이지 번역 품질이 아니다
- Teacher Forcing 환경에서 loss가 내려가도,
- 실제 추론(모델이 자기 출력을 다시 입력으로 쓰는 상황)에서는 품질이 급락할 수 있다.

3) 평가는 추론 파이프라인까지 포함해서 봐야 한다
- BLEU는 단순히 모델만의 점수가 아니라,
- 전처리/디코딩/후처리까지 포함한 전체 번역 시스템의 점수다.

---

## 다음 개선 방향 (실제로 점수/품질 올리는 쪽)

- 전처리 강화:
  - 따옴표/특수문자/불필요 기호 정리
  - 한/영 문장 페어에서 깨진 샘플 제거
- 학습 안정화:
  - LR scheduler, gradient clipping, early stopping
  - label smoothing(선택)
- 추론 개선:
  - Beam size 튜닝, length penalty 적용
  - 반복 출력 방지(no-repeat ngram 등)
- 평가 강화:
  - SacreBLEU로 재현 가능한 BLEU 계산
  - 랜덤 샘플 정성평가(“의미 보존/문법/자연스러움”) 체크리스트화


